In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import re, os, csv, pathlib, ast
import pandas as pd
from statistics import mean, variance

accent_short_forms = {"hindi":"HIN", "korean":"KOR", "vietnamese":"VTN", "arabic":"ARB", "chinese":"CHN", "spanish":"ESP"}
accent_map = {"ABA":"arabic","SKA":"arabic","YBAA":"arabic","ZHAA":"arabic",
              "BWC":"chinese","LXC":"chinese","NCC":"chinese","TXHC":"chinese",
              "ASI":"hindi","RRBI":"hindi","SVBI":"hindi","TNI":"hindi",
              "HJK":"korean","HKK":"korean","YDCK":"korean","YKWK":"korean",
              "EBVS":"spanish","ERMS":"spanish","MBMPS":"spanish","NJS":"spanish",
              "HQTV":"vietnamese","PNV":"vietnamese","THV":"vietnamese","TLV":"vietnamese"
              }
raw_string="""|ABA|M|Arabic|1129|150|\n|SKA|F|Arabic|974|150|\n|YBAA|M|Arabic|1130|149|\n|ZHAA|F|Arabic|1132|150|\n|BWC|M|Chinese|1130|150|\n|LXC|F|Chinese|1131|150|\n|NCC|F|Chinese|1131|150|\n|TXHC|M|Chinese|1132|150|\n|ASI|M|Hindi|1131|150|\n|RRBI|M|Hindi|1130|150|\n|SVBI|F|Hindi|1132|150|\n|TNI|F|Hindi|1131|150|\n|HJK|F|Korean|1131|150|\n|HKK|M|Korean|1131|150|\n|YDCK|F|Korean|1131|150|\n|YKWK|M|Korean|1131|150|\n|EBVS|M|Spanish|1007|150|\n|ERMS|M|Spanish|1132|150|\n|MBMPS|F|Spanish|1132|150|\n|NJS|F|Spanish|1131|150|\n|HQTV|M|Vietnamese|1132|150|\n|PNV|F|Vietnamese|1132|150|\n|THV|F|Vietnamese|1132|150|\n|TLV|M|Vietnamese|1132|150|"""
raw_strings=raw_string.split('\n')
gender_map={}
for lne in raw_strings:
    attrs=lne.split('|')
    gender_map[attrs[1]]=attrs[2]

composed_accent_map = {k: accent_short_forms.get(v) for k, v in accent_map.items()}

def replace_with_short_forms(s):
    for key, value in accent_short_forms.items():
        s = s.replace(key, value)
    return s

def group_speakers(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    return str(ret)

def group_accents(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    accent_counts={}
    for accent in ret:
        cnt=0
        for speaker in ret[accent]:
            cnt+=ret[accent][speaker]
        accent_counts[accent]=cnt
    return str(accent_counts)

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(group_accents, re.findall('Counter\\((.+?)\\)', s)))+list(map(group_speakers, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    return ""

def get_eta(func, eta):
    return "-n:"+str(float(eta[4:]))

In [3]:
budget = 100
# target = 50
target = 10
features = '39'
# budget, target, features = 100, 50, 'TRILL'
csv_name = "report_{}_{}_{}.csv".format(budget, target, features)

In [4]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/accent/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/random/run_1/output
# CMU_expts/speaker_without/ABA/manifests/TSS_output/within/budget_100/target_50/SKA/run_3

cols = ['speaker', 'ground', 'function', 'similarity', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-var', 'accents_run1', 'accents_run2', 'accents_run3', 'speakers_run1', 'speakers_run2', 'speakers_run3']
df = pd.DataFrame(columns = cols)

speakers = [f.name for f in os.scandir('./') if f.is_dir() and f.name != '.ipynb_checkpoints' and f.name != 'reserved_TSS_output' and f.name in accent_map]


# CMU_expts/speaker_without/ABA/manifests/TSS_output/within/budget_100/target_50/SKA/run_3
for speaker in speakers:
    # if speaker in ['ZHAA', 'MBMPS']:
    #     continue
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        print("no results for speaker {}".format(speaker))
        continue
    if 'within' not in get_dirs('./{}/manifests/TSS_output/'.format(speaker)):
        print("no within results for {}".format(accent))
        continue
    if not(pathlib.Path('./{}/manifests/TSS_output/within/budget_{}/target_{}/'.format(speaker, budget, target)).is_dir()):
        print("no results for budget {} and target {}".format(budget, target))
        continue
    for function in get_dirs('./{}/manifests/TSS_output/within/budget_{}/target_{}/'.format(speaker, budget, target)):
        if 'deprecated' in function:
            continue
        stats_file_path='./{}/manifests/TSS_output/within/budget_{}/target_{}/{}/stats.txt'.format(speaker, budget, target, function)
        try:
            stats_file = open(stats_file_path, 'r')
        except:
            continue
        lines = stats_file.readlines()
        df_selections = get_selection_counts(lines[5])
        total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
        sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
        sample_total = mean(total_selections)
        duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
        duration_total = mean(total_durations)
        df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
        df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
        try:
            wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
            wers = [x for x in wers if type(x)==float or type(x)==int]
            df_wer_mean = round(mean(wers), 2)
            df_wer_var = round(variance(wers), 3)
        except:
            print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
            continue
            wers = [0,0,0]
            df_wer_mean = 0
            df_wer_var = 999
        speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
        if function in accent_map:
            function = function+"[{}]".format(gender_map[function])
        # if function=='SKA':
        #     print("reached")
        df = df.append(dict(zip(cols, [speaker_new, "within", function, 
                        "NA", df_duration, df_samples]+ wers+[df_wer_mean, round(df_wer_var**0.5, 3)] + df_selections)), 
                       ignore_index=True)
        stats_file.close()

no WER's in file ./YBAA/manifests/TSS_output/within/budget_100/target_10/equal_random/run_1/output/test_infer_log.txt


In [5]:
df = df.sort_values(by=['speaker', 'similarity', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents_run1,accents_run2,accents_run3,speakers_run1,speakers_run2,speakers_run3
0,ABA[ARB/M],within,ABA[M],NA,358.63/358.63,92.67/92.67,22.08,21.13,22.19,21.80,0.583,{'ARB': 91},{'ARB': 95},{'ARB': 92},{'ARB': {'ABA': 91}},{'ARB': {'ABA': 95}},{'ARB': {'ABA': 92}}
1,ABA[ARB/M],within,equal_random,NA,354.85/354.85,99.33/99.33,21.78,23.39,21.20,22.12,1.134,{'ARB': 101},{'ARB': 103},{'ARB': 94},"{'ARB': {'SKA': 30, 'ZHAA': 26, 'YBAA': 24, 'A...","{'ARB': {'ZHAA': 29, 'SKA': 29, 'ABA': 23, 'YB...","{'ARB': {'SKA': 27, 'ZHAA': 25, 'YBAA': 21, 'A..."
2,ABA[ARB/M],within,other_random,NA,0.00/357.46,0.00/105.00,23.03,22.70,22.22,22.65,0.407,{'ARB': 106},{'ARB': 108},{'ARB': 101},"{'ARB': {'SKA': 39, 'ZHAA': 34, 'YBAA': 33}}","{'ARB': {'ZHAA': 44, 'YBAA': 33, 'SKA': 31}}","{'ARB': {'SKA': 36, 'YBAA': 35, 'ZHAA': 30}}"
3,ABA[ARB/M],within,random,NA,96.67/358.35,24.01/97.67,22.22,21.27,22.62,22.04,0.694,{'ARB': 95},{'ARB': 99},{'ARB': 99},"{'ARB': {'ZHAA': 25, 'ABA': 24, 'SKA': 24, 'YB...","{'ARB': {'ZHAA': 33, 'ABA': 26, 'YBAA': 23, 'S...","{'ARB': {'ZHAA': 33, 'YBAA': 27, 'ABA': 22, 'S..."
4,ASI[HIN/M],within,ASI[M],NA,357.35/357.35,123.00/123.00,12.72,13.69,13.58,13.33,0.531,{'HIN': 124},{'HIN': 122},{'HIN': 123},{'HIN': {'ASI': 124}},{'HIN': {'ASI': 122}},{'HIN': {'ASI': 123}}
5,ASI[HIN/M],within,equal_random,NA,353.12/353.12,115.67/115.67,13.30,12.97,13.80,13.36,0.418,{'HIN': 114},{'HIN': 112},{'HIN': 121},"{'HIN': {'SVBI': 32, 'ASI': 29, 'TNI': 27, 'RR...","{'HIN': {'SVBI': 32, 'ASI': 29, 'TNI': 26, 'RR...","{'HIN': {'SVBI': 34, 'ASI': 33, 'RRBI': 28, 'T..."
6,BWC[CHN/M],within,BWC[M],NA,358.96/358.96,86.00/86.00,27.11,28.12,27.52,27.58,0.508,{'CHN': 84},{'CHN': 89},{'CHN': 85},{'CHN': {'BWC': 84}},{'CHN': {'BWC': 89}},{'CHN': {'BWC': 85}}
7,BWC[CHN/M],within,equal_random,NA,351.27/351.27,90.33/90.33,29.09,31.00,33.10,31.06,2.006,{'CHN': 84},{'CHN': 96},{'CHN': 91},"{'CHN': {'BWC': 22, 'TXHC': 21, 'NCC': 21, 'LX...","{'CHN': {'TXHC': 26, 'NCC': 25, 'LXC': 23, 'BW...","{'CHN': {'TXHC': 26, 'LXC': 24, 'NCC': 22, 'BW..."
8,BWC[CHN/M],within,other_random,NA,0.00/357.87,0.00/95.00,31.30,30.10,30.74,30.71,0.601,{'CHN': 92},{'CHN': 93},{'CHN': 100},"{'CHN': {'NCC': 39, 'TXHC': 29, 'LXC': 24}}","{'CHN': {'NCC': 34, 'TXHC': 30, 'LXC': 29}}","{'CHN': {'LXC': 39, 'TXHC': 35, 'NCC': 26}}"
9,BWC[CHN/M],within,random,NA,101.67/358.38,25.70/95.33,30.18,33.06,28.64,30.63,2.244,{'CHN': 93},{'CHN': 98},{'CHN': 95},"{'CHN': {'BWC': 29, 'LXC': 28, 'TXHC': 20, 'NC...","{'CHN': {'LXC': 29, 'BWC': 26, 'TXHC': 25, 'NC...","{'CHN': {'LXC': 32, 'BWC': 22, 'NCC': 21, 'TXH..."


In [6]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# accented selection: 76 76 76 -> 76.00
# accented duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19
df2 = pd.read_csv(csv_name)
df2=df2.append(df)
df2 = df2.sort_values(by=['speaker', 'ground', 'similarity', 'function'], ascending=True, ignore_index=True)
# df2.assign(accent = df['speaker'].split('[')[1].split('/')[0] + df['two']**2).sort_values('accent')
display(df2)


,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents_run1,accents_run2,accents_run3,speakers_run1,speakers_run2,speakers_run3
0,ABA[ARB/M],all,FL1MI-n:1.0,euclidean,267.08/357.67,60.00/83.00,22.33,22.26,22.59,22.39,0.173,"{'ARB': 60, 'VTN': 8, 'ESP': 6, 'KOR': 5, 'HIN...","{'ARB': 60, 'VTN': 8, 'ESP': 6, 'KOR': 5, 'HIN...","{'ARB': 60, 'VTN': 8, 'ESP': 6, 'KOR': 5, 'HIN...","{'ARB': {'ABA': 58, 'SKA': 2}, 'VTN': {'PNV': ...","{'ARB': {'ABA': 58, 'SKA': 2}, 'VTN': {'PNV': ...","{'ARB': {'ABA': 58, 'SKA': 2}, 'VTN': {'PNV': ..."
1,ABA[ARB/M],all,FL2MI-n:1.0,euclidean,357.17/357.17,70.00/70.00,21.45,21.20,21.02,21.22,0.217,{'ARB': 70},{'ARB': 70},{'ARB': 70},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}}
2,ABA[ARB/M],all,GCMI-n:1.0,euclidean,359.83/359.83,70.00/70.00,21.71,21.53,21.78,21.67,0.130,{'ARB': 70},{'ARB': 70},{'ARB': 70},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}},{'ARB': {'ABA': 70}}
3,ABA[ARB/M],all,LogDMI-n:1.0,euclidean,345.11/358.04,76.00/80.00,21.71,21.89,21.64,21.75,0.130,"{'ARB': 76, 'ESP': 2, 'HIN': 2}","{'ARB': 76, 'ESP': 2, 'HIN': 2}","{'ARB': 76, 'ESP': 2, 'HIN': 2}","{'ARB': {'ABA': 76}, 'ESP': {'ERMS': 1, 'EBVS'...","{'ARB': {'ABA': 76}, 'ESP': {'ERMS': 1, 'EBVS'...","{'ARB': {'ABA': 76}, 'ESP': {'ERMS': 1, 'EBVS'..."
4,ABA[ARB/M],all,random,NaN,14.61/358.25,3.68/100.00,23.76,24.42,23.72,23.97,0.394,"{'KOR': 22, 'ESP': 15, 'VTN': 19, 'ARB': 20, '...","{'KOR': 20, 'CHN': 18, 'VTN': 15, 'ARB': 15, '...","{'KOR': 24, 'ESP': 16, 'ARB': 18, 'CHN': 18, '...","{'KOR': {'YDCK': 7, 'HKK': 7, 'YKWK': 4, 'HJK'...","{'KOR': {'HJK': 9, 'HKK': 5, 'YKWK': 3, 'YDCK'...","{'KOR': {'HJK': 9, 'YDCK': 5, 'HKK': 5, 'YKWK'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,ZHAA[ARB/F],all,GCMI-n:1.0,euclidean,357.43/357.43,94.00/94.00,17.84,17.77,17.73,17.78,0.055,{'ARB': 94},{'ARB': 94},{'ARB': 94},{'ARB': {'ZHAA': 94}},{'ARB': {'ZHAA': 94}},{'ARB': {'ZHAA': 94}}
170,ZHAA[ARB/F],all,LogDMI-n:1.0,euclidean,332.64/355.95,95.00/103.00,18.44,18.44,17.24,18.04,0.693,"{'ARB': 95, 'ESP': 2, 'KOR': 4, 'CHN': 1, 'VTN...","{'ARB': 95, 'ESP': 2, 'KOR': 4, 'CHN': 1, 'VTN...","{'ARB': 95, 'ESP': 2, 'KOR': 4, 'CHN': 1, 'VTN...","{'ARB': {'ZHAA': 95}, 'ESP': {'NJS': 2}, 'KOR'...","{'ARB': {'ZHAA': 95}, 'ESP': {'NJS': 2}, 'KOR'...","{'ARB': {'ZHAA': 95}, 'ESP': {'NJS': 2}, 'KOR'..."
171,ZHAA[ARB/F],all,random,NaN,20.47/358.25,6.36/100.00,17.99,18.73,18.32,18.35,0.370,"{'KOR': 22, 'ESP': 15, 'VTN': 19, 'ARB': 20, '...","{'KOR': 20, 'CHN': 18, 'VTN': 15, 'ARB': 15, '...","{'KOR': 24, 'ESP': 16, 'ARB': 18, 'CHN': 18, '...","{'KOR': {'YDCK': 7, 'HKK': 7, 'YKWK': 4, 'HJK'...","{'KOR': {'HJK': 9, 'HKK': 5, 'YKWK': 3, 'YDCK'...","{'KOR': {'HJK': 9, 'YDCK': 5, 'HKK': 5, 'YKWK'..."
172,ZHAA[ARB/F],within,ZHAA[F],NA,357.04/357.04,107.67/107.67,18.66,17.13,17.77,17.85,0.768,{'ARB': 109},{'ARB': 103},{'ARB': 111},{'ARB': {'ZHAA': 109}},{'ARB': {'ZHAA': 103}},{'ARB': {'ZHAA': 111}}


In [7]:

df2.to_csv(csv_name, index=False)
# input_file = csv.DictReader(open(csv_name))